# Week 9
> Week 9 exercises.

- toc: true
- badges: true
- comments: true
- categories: [jupyter, exercise]
- image: images/chart-preview.png

Group Members:
- Johannes Ehlers Nyholm Thomsen
- Kaare Veggerby Sandbøl
- Kasper Malmsiø Hoffmann
- Mike Holst Meldgaard

In [ ]:
import pandas as pd
import numpy as np

import os
import io
import base64
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from fastai.vision.all import *
import torch.nn.functional as nnf

import torch
torch.cuda.empty_cache()

from fastai.callback.all import *

In [ ]:
from scclib.util.image import create_samples, assemble_rgb_image, get_rgb_pieces_tensors, HPAImage

## Collect data:
Vores data, som er udleveret fra Kaggle, skal vi have samlet i det udleverede data til en kanal.
Et billede er udleveret som 4 greyscaled billeder,
hvor hvert greyscaled billede repræsenterer rød, grøn, blå og gul farve af det samlede billede.

I denne konkurrences forum har folk konkluderet, at den gule del af billedet har ingen indflydelse på modellernes præcision, hvorfor vi har valgt at sortere den fra.

Vi skal derfor have samlet de 3 RBG billeder til et samlet billede.
Til at gøre dette har vi lavet et hjælpe bibliotek, da det bare er en masse python kode.
Koden tager de 3 billeder, putter deres data i den respektive farve kanal for et samlet billede og resizer til en ønsket størrelse.
Vi resizer allerede her, da vi så kan gøre det på GPU i stedet for CPU'en.

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
data_path = Path('Z:\cloud\data\hpa-single-cell-image-classification')
path = Path('../data')

In [ ]:
df = pd.read_csv(data_path/'train.csv')

In [ ]:
df.head()

In [ ]:
labels = [str(i) for i in range(19)]
for x in labels:
    df[x] = df['Label'].apply(lambda r: int(x in r.split('|')))

df.head()

In [ ]:
dfs_0 = df[df['Label'] == '0'].sample(n=300, random_state=42).reset_index(drop=True)
dfs_1 = df[df['1'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_1u = df[df['Label'] == '1'].sample(n=221, random_state=42).reset_index(drop=True)
dfs_2 = df[df['Label'] == '2'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_3 = df[df['Label'] == '3'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_4 = df[df['Label'] == '4'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_5 = df[df['Label'] == '5'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_6 = df[df['6'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_7 = df[df['Label'] == '7'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_8 = df[df['Label'] == '8'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_9 = df[df['9'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_9u = df[df['Label'] == '9'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_10 = df[df['10'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_10u = df[df['Label'] == '10'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_11 = df[df['11'] == 1].reset_index(drop=True)
dfs_12 = df[df['Label'] == '12'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_13 = df[df['Label'] == '13'].sample(n=400, random_state=42).reset_index(drop=True)
dfs_14 = df[df['Label'] == '14'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_15 = df[df['15'] == 1].reset_index(drop=True)
dfs_16 = df[df['Label'] == '16'].sample(n=350, random_state=42).reset_index(drop=True)
dfs_17 = df[df['17'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_18 = df[df['18'] == 1].reset_index(drop=True)
dfs_ = [dfs_0, dfs_1, dfs_1u, dfs_2, dfs_3, dfs_4, dfs_5, dfs_6, dfs_7, dfs_8, dfs_9, dfs_9u, dfs_10, dfs_10u,
        dfs_11, dfs_12, dfs_13, dfs_14, dfs_15, dfs_16, dfs_17, dfs_18]

In [ ]:
dfs = pd.concat(dfs_, ignore_index=True)
dfs.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
data_path = Path('../data')

image_path = Path('Z:/cloud/data/hpa-single-cell-image-classification/train')
image_destination = data_path/'train'

if not image_destination.exists():
    os.makedirs(image_destination)

In [ ]:
example_image = image_path.ls()[0].name
image_id, image_ext = example_image.split('.')

image_id = image_id.split('_')[0]

## Data Preparation:
Billederne bliver udleveret i en kvalitet af 1080p og op efter.
Vi har valgt at resize dem til 512 gange 512 for at bevare nok billede kvalitet men også undgå at løbe tør for hukkomelse på computeren.

Efter vi har fundet dataen skal vi have loaded dem ind i en dataloader.
Samtidig med at vi gør det cropper vi også billederne ned til 256 gange 256 opløsning,
og da vi bruger batch_tfms sker der en tilfældig crop og augmentering.
Der sker dog kun resizing på valideringssettet.

Vi har altså været nød til selv manuelt at implementere den fremgangsmåde,
som fastai kalder presizing, da vores udleverede billeder er i så høj kvalitet.
Vi interpolerer dog 2 gange i stedet for kun én gang, som hvis vi brugte fastai presizing,
hvilket resulterer i en ringere billedkvalitet, men vi vinder performance.

In [ ]:
new_img = HPAImage(id=image_id,
                   ext=image_ext,
                   path=image_path,
                   device="cuda:0",
                   size=512)

new_img.plot_all()

In [ ]:
create_samples(512, dfs, image_path, image_destination, csv_path=data_path/'cells_sample.csv', device="cuda:0")

In [ ]:
model_path = Path('../models')
path = Path('../data')

In [ ]:
cell_df = pd.read_csv(path/'cells_sample.csv')
cell_df.head()

In [ ]:
def get_x(r): return path/'train'/f"{r['image_id']}.png"
def get_y(r): return r['image_labels'].split('|')

In [ ]:
def get_dls(bs: int, size: int):
    return DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                     splitter=RandomSplitter(seed=42),
                     get_x=get_x,
                     get_y=get_y,
                     batch_tfms=aug_transforms(size=size, min_scale=0.75))\
        .dataloaders(cell_df, bs=bs, num_workers=0)

In [ ]:
batch_size = 64
image_size = 256

dls = get_dls(batch_size, image_size)

In [ ]:
dls.show_batch()

## Choose Model:
Vi har valgt at bruge et residualt netværk, som er trænet på ImageNet.
I vores model er det kun et netværk med 18 lag, så vi ikke skal vente hundrede år mellem hver træning,
når vi eksperimenterer med modellen.
Derudover så vi ikke en markant højere præcision, da vi trænede med modellen med 50 lag.
Det kan også ses, at vi bruger fp16 for at spare regnekraft mens vi træner.

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=partial(accuracy_multi, thresh=0.2)).to_fp16()


## Evaluation:
Til vores models metric bruger vi standard præcision for multicategory, hvilket er accuracy_multi.
Vi vil dog gerne have en threshhold på 0.2, derfor er vi nød til at give mertics en partial,
hvilket betyder at der laves en ny version af accuracy_multi, hvor threshold er den værdi vi sender med i partial.

## Tuning:
Da vi har valgt at bruge en på-forhånd trænet model, ResNet, er vi nød til at finjustere den til vores problem.
Dette gøres ved at kalde fine_tune på vores learner.
fine_tune smider det sidste linære lag i modellen væk og skifter det ud med et nyt linært lag.
Dernæst fryser modellen vægtene i alle de gamle lag, og træner nu det nye linære lag i en epoche.
Derefter tøes de frosne lag op og modellen trænes nu sammen i det ønskede antal gange - her er det 5.

På denne måde undgår vi at smide alt den træning modellen har fra ImageNet væk,
og modellen bliver stadig optimeret til lige vores problemstilling.

In [ ]:
learn.lr_find()

In [ ]:
with learn.no_bar():
    learn.fine_tune(5, base_lr=0.03)

In [ ]:
class Hook:
    def hook_func(self, model, input, output):
        self.stored = output.detach().clone()

In [ ]:
hook_output = Hook()
hook = learn.model[0].register_forward_hook(hook_output.hook_func)

In [ ]:
with torch.no_grad(): output = learn.model.eval()(x)

In [ ]:
act = hook_output.stored[0]

In [ ]:
F.softmax(output, dim=-1)

In [ ]:
dls.vocab

In [ ]:
cam_map = torch.einsum('ck,kij->cij', learn.model[1][-1].weight, act)
cam_map.shape

In [ ]:
cam_map[1]

In [ ]:
x = dls.one_batch()[0][0].unsqueeze(0)

In [ ]:
print(x.shape)
print(x)

In [ ]:
x_dec = TensorImage(dls.train.decode((x,))[0][0])
_,ax = plt.subplots()
x_dec.show(ctx=ax)
ax.imshow(cam_map[1].detach().cpu(), alpha=0.5, extent=(0,256,256,0), interpolation='bilinear', cmap='nipy_spectral')

In [ ]:
path = Path('Z:/cloud/data/hpa-single-cell-image-classification/test')

In [ ]:
images = get_images_files(path)
images

## Prediction:
Vi har ikke kunnet få Kaggle til at give os en score på vores model,
da de vil vores predictions i en meget speciel og træls opsætning.

In [ ]:
test_dl = learn.dls.test.test_dl(images)

In [ ]:
with learn.no_bar():
    predictions, _ = learn.predict(dl=test_dl)